# Check result

In [1]:
import pandas as pd
train_file = pd.read_parquet("./data/solfile/train_file.parquet", engine="fastparquet")
train_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167310 entries, 0 to 167309
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   file_name         167310 non-null  object
 1   file_address      167310 non-null  object
 2   source_code       167310 non-null  object
 3   abi               167310 non-null  object
 4   compiler_version  167310 non-null  object
 5   library           167310 non-null  object
dtypes: object(6)
memory usage: 7.7+ MB


In [2]:
test_file = pd.read_parquet("./data/solfile/test_file.parquet", engine="fastparquet")
test_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18594 entries, 0 to 18593
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   file_name         18594 non-null  object
 1   file_address      18594 non-null  object
 2   source_code       18594 non-null  object
 3   abi               18594 non-null  object
 4   compiler_version  18594 non-null  object
 5   library           18594 non-null  object
dtypes: object(6)
memory usage: 871.7+ KB


# Experiment

In [3]:
versions = list(set(train_file["compiler_version"].tolist() + test_file["compiler_version"].tolist()))
versions = list(set([version.split('+')[0][1:].split('-')[0] for version in versions]))
string = ""
for version in versions:
    string = string + '{' + f'version: "{version}"'+ '},\n'
with open("sol_version.txt", "w") as f:
    f.write(string)

In [4]:
from subprocess import run 

for i in range(len(test_file)):
    source = test_file.loc[i, "source_code"]
    with open("./hardhat/contracts/sample.sol", "w") as f:
        f.write(source)
    cmd = """
    cd hardhat
    npx hardhat compile
    """
    data = run(cmd, capture_output=True, shell=True, text=True)
    # print(data.stdout)
    # print("-" * 400)
    # print(data.stderr)
    # print(test_file.loc[i, "file_address"])
    if 'Compiled 1 Solidity file successfully' in data.stdout:
        with open("compilable.txt", "a") as f:
            f.write(f"{i}\n---------------------------------------------------\n")
    

KeyboardInterrupt: 